In [126]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.express as px

In [127]:
DATA_PATH = '../data/GEFCom2014Data/Wind/raw_data_incl_features.csv'

#def update_data(day):
## read data
data = pd.read_csv('../data/GEFCom2014Data/Wind/raw_data_incl_features.csv', parse_dates=['TIMESTAMP']) #, index_col='TIMESTAMP')
data.interpolate(method='bfill', inplace=True)
# data.info()
# data.head()


In [128]:
day = '2013-07-15'

data_forecast = pd.read_csv('RandomForest_Predictions.csv', parse_dates=['TIMESTAMP'])
features_forecast = ['ZONEID', 'TIMESTAMP', 'HOUR', 'WEEKDAY', 'WS100', 'WD100']
data_all = pd.read_csv('../data/GEFCom2014Data/Wind/raw_data_incl_features.csv', parse_dates=['TIMESTAMP'])

## get the required data
def get_data(day, 
             data,
             features=None,
             delta_days=None):
    day +=' 1:00:00'
    time_start = pd.to_datetime(day)
    time_end = time_start + pd.Timedelta(value=23, unit='hour')
    if delta_days is not None:
        time_delta = pd.Timedelta(value=1, unit='day')
        time_start += time_delta*delta_days
        time_end += time_delta*delta_days
    data = data[(data['TIMESTAMP']>=time_start) & (data['TIMESTAMP']<=time_end)]
    if features is not None and len(features)>0:
        if all(x in data.columns for x in features):
            data = data[features]
    return data

In [129]:
def add_HOUR_column(df):
    if 'HOUR' not in df.columns and 'TIMESTAMP' in df.columns:
        df['HOUR'] = df['TIMESTAMP'].dt.hour
    

In [177]:
PATH_PREDICTIONS = 'RandomForest_Predictions.csv'
PATH_DATA_ALL = 'raw_data_incl_features.csv'
data_forecast = pd.read_csv(PATH_PREDICTIONS, parse_dates=['TIMESTAMP'])
add_HOUR_column(data_forecast)
data_retro = pd.read_csv(PATH_DATA_ALL, parse_dates=['TIMESTAMP'])
add_HOUR_column(data_retro)
#data_forecast.info()

In [192]:
dff = get_data(day, data_forecast)
dff_yesterday = get_data(day, data_retro, features=['ZONEID', 'TIMESTAMP', 'TARGETVAR'], delta_days=-1)
dff_yesterday = dff_yesterday.groupby('ZONEID').mean().reset_index()
dff_yesterday['ZONEID'] = ['Zone '+str(ii) for ii in range(1,11)]
dff_yesterday

,ZONEID,TARGETVAR
0,Zone 1,0.075101
1,Zone 2,0.321632
2,Zone 3,0.396583
3,Zone 4,0.321204
4,Zone 5,0.473059
5,Zone 6,0.558990
6,Zone 7,0.089144
7,Zone 8,0.110136
8,Zone 9,0.267250
9,Zone 10,0.354470


In [171]:
colors = px.colors.qualitative.Plotly
color_dict = {'Zone '+str(z): c for z,c in zip(range(1,11), colors)}
from PIL import ImageColor
color_dict_light = { k:ImageColor.getcolor(v, "RGB") for k,v in color_dict.items()}
color_dict_light = { k:"rgba({}, {}, {}, {})".format(v[0], v[1], v[2], 0.4) for k,v in color_dict_light.items()}


#color_dict_light

In [187]:
selected_zone = ['Zone '+str(ii) for ii in range(1,11)]
df_forecast = dff


selected_zone = sorted(selected_zone, key=lambda x : x[-2:])
df_forecast = df_forecast[selected_zone]
df = pd.DataFrame(df_forecast.mean())
df

,0
Zone 1,0.048209
Zone 2,0.088525
Zone 3,0.030019
Zone 4,0.062117
Zone 5,0.096730
Zone 6,0.097216
Zone 7,0.023915
Zone 8,0.025530
Zone 9,0.019897
Zone 10,0.113885


In [200]:
dff = get_data(day, data_forecast)
dff_yesterday = get_data(day, data_retro, features=['ZONEID', 'TIMESTAMP', 'TARGETVAR'], delta_days=-1)
dff_yesterday = dff_yesterday.groupby('ZONEID').mean()
dff_yesterday.index = ['Zone '+str(ii) for ii in range(1,11)]
dff_yesterday

,TARGETVAR
Zone 1,0.075101
Zone 2,0.321632
Zone 3,0.396583
Zone 4,0.321204
Zone 5,0.473059
Zone 6,0.558990
Zone 7,0.089144
Zone 8,0.110136
Zone 9,0.267250
Zone 10,0.354470


In [356]:
def get_figure_cumulated_energy(df_forecast, df_yesterday, selected_zone):
    selected_zone = sorted(selected_zone, key=lambda x : x[-2:])
    # forecast data
    df_forecast = pd.DataFrame(df_forecast[selected_zone].mean())
    df_forecast.columns = ['TARGETVAR']
    # yesterday's data
    df_yesterday = df_yesterday.groupby('ZONEID').mean()
    
    bars = []
    for zone in selected_zone[0:]:
        bars.append( 
            go.Bar( 
                x=[zone, zone+' yesterday'], 
                y=[df_forecast.loc[zone]['TARGETVAR'], df_yesterday.loc[zone]['TARGETVAR']],
                marker={'color': [color_dict[zone], color_dict_light[zone]]}
            )
        )
    fig = go.Figure(data=bars)
    fig.update_yaxes(range = [0,1])
    fig.layout.template = 'plotly_white'
    fig.update_layout( 
            title='Forecasted Energy Output over the whole day')
    fig.layout.showlegend = False
    print(range(0,2*len(selected_zone)))
    fig.update_layout(
        xaxis = dict(
        tickmode = 'array',
        tickvals = list(range(0,2*len(selected_zone))),
        ticktext = [selected_zone[ii//2] if ii%2==0 else '' for ii in range(0,2*len(selected_zone))]
        )
    )
    return fig

In [360]:
selected_zone = ['Zone '+str(ii) for ii in range(1,11)]
df_forecast = dff

dff = get_data(day, data_forecast)
df_yesterday = get_data(day, data_retro, features=['ZONEID', 'TIMESTAMP', 'TARGETVAR'], delta_days=-1)
df_yesterday['ZONEID'] = df_yesterday['ZONEID'].apply(lambda x: 'Zone '+str(x))

selected_zone = sorted(selected_zone, key=lambda x : x[-2:])
fig = get_figure_cumulated_energy(df_forecast, df_yesterday, selected_zone);
fig.show()

range(0, 20)


In [361]:
data = pd.read_csv('../data/GEFCom2014Data/Wind/raw_data_incl_features.csv', parse_dates=['TIMESTAMP']) #, index_col='TIMESTAMP')
data.interpolate(method='bfill', inplace=True)

In [366]:
day = '2013-06-30'
day +=' 1:00:00'
time_start = pd.to_datetime(day)
time_end = time_start + pd.Timedelta(value=23, unit='hour')
data = data[(data['TIMESTAMP']>=time_start)]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44400 entries, 13104 to 175439
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ZONEID      44400 non-null  int64         
 1   TIMESTAMP   44400 non-null  datetime64[ns]
 2   TARGETVAR   44394 non-null  float64       
 3   U10         44400 non-null  float64       
 4   V10         44400 non-null  float64       
 5   U100        44400 non-null  float64       
 6   V100        44400 non-null  float64       
 7   HOUR        44400 non-null  int64         
 8   MONTH       44400 non-null  int64         
 9   WEEKDAY     44400 non-null  int64         
 10  IS_HOLIDAY  44400 non-null  int64         
 11  WS10        44400 non-null  float64       
 12  WS100       44400 non-null  float64       
 13  WD10        44400 non-null  float64       
 14  WD100       44400 non-null  float64       
 15  WD100CARD   44400 non-null  object        
 16  WD10CARD    44400

In [367]:
data.to_csv('raw_data_incl_features_test.csv', index=False)